In [119]:
import pandas as pd
import os
import csv
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


In [120]:
from Experimentos import Experimentos

In [121]:
from scipy.ndimage.filters import median_filter

In [122]:
for i, exp in enumerate(Experimentos):
    print(i, exp, (exp.flir is not None))

0 5cc50C.csv False
1 1cc54C.csv False
2 25cc54C.csv False
3 10cc60C.csv False
4 25cc60C.csv False
5 1cc59C.csv False
6 1cc47C.csv True
7 1cc55C.csv True
8 25cc58C.csv True
9 25cc70C.csv True
10 10cc67C.csv True
11 10cc59C.csv True
12 5cc63C.csv False
13 5cc59C.csv True
14 10cc49C.csv True
15 25cc62C.csv False
16 1cc62C.csv False
17 5cc53C.csv True
18 5a25cc56C.csv True
19 5cc57C.csv True
20 25cc50C.csv True
21 10cc55C.csv True
22 1cc24C.csv False
23 10cc57C.csv True
24 10cc24C.csv False
25 5cc55C.csv True
26 5cc58C.csv True
27 1cc48C.csv True
28 1cc51C.csv True
29 5cc48C.csv True
30 10cc50C.csv True
31 1cc54,7C.csv True
32 1cc52C.csv True


In [128]:
Experimentos[30]

10cc50C.csv

In [95]:
numeropixeles = 1020100
rho_manjar = 1340
kilos_manjar = 2
new_row = Experimentos[30].flir['FLIR: area']/numeropixeles

In [96]:
for exp in Experimentos:
    if exp.flir is not None:
        try:
            exp.flir[(np.abs(stats.zscore(df)) < 3).all(axis=1)]
        except:
            continue

In [98]:
Experimentos[30].flir = Experimentos[30].flir[median_filter(Experimentos[30].flir['FLIR: area'], size=50)]

IndexError: indices are out-of-bounds

In [100]:
Experimentos[30].flir= median_filter(Experimentos[30].flir['FLIR: area'], size=5).shape

TypeError: tuple indices must be integers or slices, not str

In [101]:
#for exp in Experimentos:
#   if exp.flir is not None:
#      exp.flir = median_filter(exp.flir['FLIR: area'], size=50)
# else:
#    continue


In [102]:
for exp in Experimentos:
    if exp.flir is not None:
        try:
            exp.flir['area_m2'] = exp.flir['FLIR: area']/numeropixeles
            exp.flir['volumen_emitido'] = exp.param['caudal'][0]*exp.flir['FLIR: tiempo']
        except:
            continue
        volumen_total = kilos_manjar/rho_manjar
        exp.flir['volumen_emitido'][exp.flir['volumen_emitido'] > volumen_total] = volumen_total
        exp.flir['espesor'] = 3*exp.flir['volumen_emitido']/(2*exp.flir['area_m2'])
        

C:\Users\CCVV\Anaconda3b\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [64]:
# interpolate + smooth
itp = interp1d(x,y, kind='linear')
window_size, poly_order = 101, 3
yy_sg = savgol_filter(itp(xx), window_size, poly_order)


# or fit to a global function
def func(x, A, B, x0, sigma):
    return A+B*np.tanh((x-x0)/sigma)

fit, _ = curve_fit(func, x, y)
yy_fit = func(xx, *fit)

fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(x, y, 'r.', label= 'Unsmoothed curve')
ax.plot(xx, yy_fit, 'b--', label=r"$f(x) = A + B \tanh\left(\frac{x-x_0}{\sigma}\right)$")
ax.plot(xx, yy_sg, 'k', label= "Smoothed curve")
plt.legend(loc='best')

NameError: name 'x' is not defined

In [103]:
#funciona para plotear espesor en el tiempo
plt.scatter(Experimentos[30].flir['FLIR: tiempo'], Experimentos[23].flir['espesor'], linestyle='-')

TypeError: tuple indices must be integers or slices, not str

In [104]:
def rango_temperatura(temp):
    if temp < 40:
        return 1
    if temp < 45:
        return 2
    if temp < 50:
        return 3
    if temp < 55:
        return 4
    if temp < 60:
        return 5
    if temp < 65:
        return 6
    if temp < 70:
        return 7

In [105]:
listaporcaudal = []

for exp in Experimentos:
    if exp.flir is not None:
        try:
            holder = {} #es un diccionario 
            holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
            holder['espesor'] = exp.flir['espesor']#toda la columna
            holder['tiempo'] = exp.flir['FLIR: tiempo']#toda la columna
            holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
            holder['angulo'] = exp.param['angulo inclinacion'][0]
            holder['uso'] = exp.param['uso manjar'][0]

            listaporcaudal.append(holder)
        except:
            continue

In [106]:
#se arman grupos segun el valor del caudal
from itertools import groupby
count = 0

#key es la lista donde estan los diccionarios de exp y DF
#group es que se agrupa segun el valor de 'caudal' en este caso

for key, group in groupby(sorted(listaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    count += 1
    #for exp in group:
        #muestra el nombre del grupo y los elementos que tiene
        #print(exp['nombre']) 

1e-06
5e-06


In [107]:
fig = plt.figure()
axes = [fig.add_subplot(3,2,i+1) for i in range(count)]
counter = 0



for key, group in groupby(sorted(listaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    ax = axes[counter]
    #ax.set_xlim(minimoejelargo, maximoejelargo)
    #ax.set_ylim(0, maximoejeespesor)
    ax.set_title('{}cc'.format(key*1000000))
    counter += 1
    linea1= ax.plot([],[], label='15°', linestyle='--', color='black')
    linea2= ax.plot([],[], label='10°', color='black')
    
    for exp in group:
        #print("C{}".format(exp["rango temperatura"]))
        #if exp['uso'] == 1: 
            if exp['angulo'] == 15:
                exp.flir[(np.abs(stats.zscore(exp.flir)) < 3).all(axis=1)]
                ax.plot(exp['tiempo'], exp['espesor'], linestyle= '--')
            else:
                exp.flir[(np.abs(stats.zscore(exp.flir)) < 3).all(axis=1)]
                ax.plot(exp['tiempo'], exp['espesor'])
        
#legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
#plt.gca().add_artist(legend1)
fig.show()

1e-06


AttributeError: 'dict' object has no attribute 'flir'

In [55]:
colors = ['r', 'b', 'g', 'y', 'c', 'k', 'black', 'brown', 'orange', 'purple', 'violet']

In [68]:
#flir['FLIR: rango 1']*flir['FLIR: area']

In [66]:
axp = flir.plot.scatter('FLIR: tiempo', 'FLIR: area', color=colors[0])

for i, col in enumerate(flir):
    print(i, flir[col][0])
    if i == 0 or i ==1:
        continue
    flir['p'] = flir['FLIR: area']*flir[col]
    flir.plot.area('FLIR: tiempo', 'p', ax= axp, color=colors[i])

#flir.plot.scatter('FLIR: tiempo', 'FLIR: area', 'FLIR: rango 1', 'FLIR: rango 2')

NameError: name 'flir' is not defined

In [77]:
flir[['FLIR: rango 1', 'FLIR: rango 2', 'FLIR: rango 3']].plot.area()